### Validation of Tetrahedral Projection

This notebook is a set of tests to ensure that the tetrahedral partial parcel membership make sense.

**Context:**
- For simplicity in testing we'll use an identity affine matrix so that tetrahedral membership resolution in MR and RAS space is equivalent]
***

1. Tetrahedron grid test -- we systematically place tetrahedrons at various voxel boundaries with known partial memberships (by known it means 

2. Tetrahedral visualization and cross-validation with voxel membership -- visualize voxel placement and estimated membership. 

In [ ]:
from scipy.spatial import ConvexHull
import nibabel as nib
import numpy as np
from nilearn import plotting, image
import matplotlib.pyplot as plt

In [ ]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

In [ ]:
#https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Importing%20Notebooks.html
#This is for loading functions for another jupyter notebook
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        
        
        path = os.path.join(self.path,fullname) + '.ipynb'

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.cells:
                if cell.cell_type == 'code':
                    # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.source)
                    # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [13]:
tlib_path = '/projects/jjeyachandra/rtms_optimize/notebooks/python2/'
nbload = NotebookLoader(tlib_path)
tetrapro = nbload.load_module('tetrapro')

importing Jupyter notebook from /projects/jjeyachandra/rtms_optimize/notebooks/python2/tetrapro.ipynb


### Tetrahedron Grid Test 
Define an NxN array of pseudo-voxels. Then define several classes of tetrahedrons:
1. Fully embedded tetrahedrons -- expect full voxel ownership of tetrahedron
2. Horizontal/Vertically bordered tetrahedrons -- expect full ownership of tetrahedron
3. Horizontal/Vertically exclusive crossing -- expect 2 voxel ownership of tetrahedron
4. Multi-voxel ownership -- expect 3-4 voxel ownership of tetrahedron within adjacent voxel groups

In [ ]:
#Create fake inputs
